# Import

In [1]:
# 필요한 라이브러리 임포트

import pandas as pd
import random
import os
import numpy as np

import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
# 시드값 고정을 위한 클래스 정의.

class CFG:
    SEED = 42

In [3]:
# 시드값 호출을 위한 함수 정의.

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

# Data Load

In [4]:
# 학습 데이터와 테스트 데이터 불러오기.

train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test.csv')

In [7]:
# trait과 class 피쳐의 특징 파악.

print(train['trait'].value_counts())
print('--------------')

print(train['class'].value_counts())
print('--------------')

2    193
1     69
Name: trait, dtype: int64
--------------
B    114
C     79
A     69
Name: class, dtype: int64
--------------


In [5]:
# 학습 데이터와 테스트 데이터를 동일한 형태로 만들어 주기 위해 전처리 가능한 함수 정의.

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [6]:
# 함수를 이용해서 학습 데이터와 테스트 데이터 전처리 진행.

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

# Data Pre-processing
### Label-Encoding

In [7]:
# 머신러닝 학습을 위한 문자열 데이터를 숫자 데이터로 변경하기 위해 LabelEncode() 선언.

class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()

# 유전체의 염기서열을 담고 있는 칼럼들을 snp_col의 이름으로 지정.

snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]
print(snp_col)

['SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09', 'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15']


In [8]:
# snp_col 칼럼의 데이터들을 snp_data 이름의 배열에 저장.

snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

In [17]:
# 문자열 데이터들을 모두 숫자 데이터로 변경.

train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [18]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

# Model Fit

In [48]:
k = 1
clf = KNeighborsClassifier(n_neighbors=k).fit(train_x, train_y)
print(clf)

KNeighborsClassifier(n_neighbors=1)


In [49]:
yhat = clf.predict(test_x)
print("Train set Accuracy :", metrics.accuracy_score(train_y, clf.predict(train_x)))

Train set Accuracy : 1.0


-. k = 1로 예측한 결과를 제출하면 0.9257142857의 결과를 얻는다.

In [50]:
k = 2
clf = KNeighborsClassifier(n_neighbors=k).fit(train_x, train_y)
print(clf)

KNeighborsClassifier(n_neighbors=2)


In [51]:
yhat = clf.predict(test_x)
print("Train set Accuracy :", metrics.accuracy_score(train_y, clf.predict(train_x)))

Train set Accuracy : 0.9312977099236641


-. k = 2로 예측한 결과를 제출하면 0.9320261438의 결과를 얻는다.

In [43]:
k = 3
clf = KNeighborsClassifier(n_neighbors=k).fit(train_x, train_y)
print(clf)

KNeighborsClassifier(n_neighbors=3)


In [44]:
yhat = clf.predict(test_x)
print("Train set Accuracy :", metrics.accuracy_score(train_y, clf.predict(train_x)))

Train set Accuracy : 0.9427480916030534


-. k = 3로 예측한 결과를 제출하면 0.9622367466의 결과를 얻는다.  
-. 이는 처음의 베이스라인 코드를 통한 결과를 제출했을 때와 같은 수치이다.

In [52]:
k = 4
clf = KNeighborsClassifier(n_neighbors=k).fit(train_x, train_y)
print(clf)

KNeighborsClassifier(n_neighbors=4)


In [53]:
yhat = clf.predict(test_x)
print("Train set Accuracy :", metrics.accuracy_score(train_y, clf.predict(train_x)))

Train set Accuracy : 0.9427480916030534


(1부터 1씩 증가하면서 해봤는데 역시나 3이 최적이었음. 4부터 바로 떨어짐.) - 대략 0.96에서 0.94로 떨어졌음.

# Submission

In [45]:
submit = pd.read_csv('open/sample_submission.csv')

In [46]:
submit['class'] = class_le.inverse_transform(yhat)

In [47]:
submit.to_csv('./answer/submit.csv', index=False)